In [ ]:
# This is a project to use the Py-Coin-Gecko Python Client/Wrapper to access the Coin Gecko API, updated every minute by Coin-Gecko.
#In this function we are getting data on bitcoin, in U.S. Dollars, for the past 30 days and creating a candlestick graph displaying 
# the  minimum, maximum, first, and last price of each day across 24 observations per day (one per hour), grouped by date.

In [3]:
!pip install plotly
!pip install pycoingecko
!pip install mplfinance

In [2]:
import plotly.graph_objects as go
import plotly.offline as plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [4]:
cg = CoinGeckoAPI()

In [5]:
# id is the name of the coin we want,
# vs_currency is the currency we want the price in,
# and days is how many days back from today we want
bitcoin_data = cg.get_coin_market_chart_by_id(id ='bitcoin', vs_currency = 'usd', days = 30) 

In [6]:
bitcoin_data

{'prices': [[1624744855286, 31675.775039555298],
  [1624748419397, 31870.282842702174],
  [1624752016717, 32023.973901774916],
  [1624755686096, 32331.880445318537],
  [1624759277676, 32987.11329765951],
  [1624762855275, 33049.60071727176],
  [1624766498911, 33271.709049034645],
  [1624770091119, 33273.359102643706],
  [1624773682025, 32873.65414378919],
  [1624777291228, 33207.6972809356],
  [1624781455080, 33500.249918432855],
  [1624784402893, 33250.598083274905],
  [1624788042797, 32665.113744341408],
  [1624791719845, 32762.79839183285],
  [1624795322872, 33413.15774616591],
  [1624798935238, 33430.657487662764],
  [1624802432213, 33095.520085215125],
  [1624806091984, 33304.882466727606],
  [1624809731417, 33229.71107407361],
  [1624813256546, 33019.170617543365],
  [1624816831440, 33153.30288407903],
  [1624820477495, 32815.93601477983],
  [1624824075563, 32989.537428752796],
  [1624827626180, 32767.219874254646],
  [1624831305882, 32890.84546346474],
  [1624834914218, 34506.09

In [7]:
type(bitcoin_data)

dict

In [8]:
# The response we get is a JSON document 
# consisting of prices, market caps, and total volumes along with timestamps for each observation
# but we are focussed only on the prices 
bitcoin_price_data = bitcoin_data["prices"]

In [9]:
bitcoin_price_data[:5]

[[1624744855286, 31675.775039555298],
 [1624748419397, 31870.282842702174],
 [1624752016717, 32023.973901774916],
 [1624755686096, 32331.880445318537],
 [1624759277676, 32987.11329765951]]

In [10]:
# Turn into pandas dataframe
data = pd.DataFrame(bitcoin_price_data, columns = ["Timestamp", "Price"])

In [11]:
data[:5]

,Timestamp,Price
0,1624744855286,31675.775040
1,1624748419397,31870.282843
2,1624752016717,32023.973902
3,1624755686096,32331.880445
4,1624759277676,32987.113298


In [12]:
# convert timestamp to date and store it as a column called Date
data["Date"] = data["Timestamp"].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [13]:
data[:5]

,Timestamp,Price,Date
0,1624744855286,31675.775040,2021-06-26
1,1624748419397,31870.282843,2021-06-26
2,1624752016717,32023.973902,2021-06-26
3,1624755686096,32331.880445,2021-06-26
4,1624759277676,32987.113298,2021-06-26


In [14]:
data.drop("Timestamp",1)

,Price,Date
0,31675.775040,2021-06-26
1,31870.282843,2021-06-26
2,32023.973902,2021-06-26
3,32331.880445,2021-06-26
4,32987.113298,2021-06-26
...,...,...
716,39240.536698,2021-07-26
717,39841.370269,2021-07-26
718,39580.479489,2021-07-26
719,37587.103371,2021-07-26


In [15]:
candlestick_data = data.groupby(data.Date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})

In [17]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['Date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()